In [1]:
# -*- coding: utf-8 -*-
# VGG模型，GPU模式2

import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

# from model import *
# 准备数据集
from torch import nn
from torch.utils.data import DataLoader

# 定义训练的设备
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

train_data = torchvision.datasets.CIFAR10(root="./dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="./dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)

# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
# 如果train_data_size=10, 训练数据集的长度为：10
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))


# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 创建网络模型


class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


vgg = VGG()
vgg = vgg.to(device)


# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.to(device)
# 优化器
# learning_rate = 0.01
# 1e-2=1 x (10)^(-2) = 1 /100 = 0.01
learning_rate = 1e-2
optimizer = torch.optim.SGD(vgg.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 100

# 添加tensorboard
writer = SummaryWriter("./logs_train")

for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))

    # 训练步骤开始
    vgg.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = vgg(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    vgg.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = vgg(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy /
                      test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(
    #     vgg, "/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/Pytorch/demo1/vggmodel/vgg_{}.pth".format(i))
    print("模型已保存")

writer.close()


Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-------第 1 轮训练开始-------


/home/tzr/anaconda3/envs/RockLabTorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


训练次数：100, Loss: 2.2887492179870605
训练次数：200, Loss: 2.2761948108673096
训练次数：300, Loss: 2.234231948852539
训练次数：400, Loss: 2.1584043502807617
训练次数：500, Loss: 2.081246852874756
训练次数：600, Loss: 2.0152499675750732
训练次数：700, Loss: 2.0384020805358887
整体测试集上的Loss: 321.500901222229
整体测试集上的正确率: 0.26089999079704285
模型已保存
-------第 2 轮训练开始-------
训练次数：800, Loss: 1.9140576124191284
训练次数：900, Loss: 1.921062707901001
训练次数：1000, Loss: 1.9746365547180176
训练次数：1100, Loss: 2.023158550262451
训练次数：1200, Loss: 1.770092248916626
训练次数：1300, Loss: 1.7552846670150757
训练次数：1400, Loss: 1.8005107641220093
训练次数：1500, Loss: 1.8647067546844482
整体测试集上的Loss: 310.72112929821014
整体测试集上的正确率: 0.29729998111724854
模型已保存
-------第 3 轮训练开始-------
训练次数：1600, Loss: 1.8577834367752075
训练次数：1700, Loss: 1.6639981269836426
训练次数：1800, Loss: 1.937606692314148
训练次数：1900, Loss: 1.7364317178726196
训练次数：2000, Loss: 1.9710673093795776
训练次数：2100, Loss: 1.5448391437530518
训练次数：2200, Loss: 1.512139081954956
训练次数：2300, Loss: 1.800081729888916
整体测

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)